# 👀 Multilayer perceptron (MLP)

In this notebook, we'll walk through the steps required to train your own multilayer perceptron on the CIFAR dataset

In [ ]:
import os, sys
from dotenv import load_dotenv

load_dotenv()
python_path = os.getenv('PYTHONPATH')
data_path = os.getenv('DATA_PATH')
if python_path:
    for path in python_path.split(os.pathsep):
        if path not in sys.path:
            sys.path.append(path)

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from tqdm import tqdm, trange

from notebooks.pt_utils import display, Trainer

## 0. Parameters <a name="parameters"></a>

In [ ]:
NUM_CLASSES = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 1. Prepare the Data <a name="prepare"></a>

In [ ]:
cifar10_transform = transforms.ToTensor()
train_dataset = datasets.CIFAR10(root=data_path, train=True, transform=cifar10_transform)
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [0.8, 0.2])

test_dataset = datasets.CIFAR10(root=data_path, train=False, transform=cifar10_transform)

In [ ]:
BATCH_SIZE = 256
NUM_WOERKERS = 24

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WOERKERS, pin_memory=True, pin_memory_device='cuda')
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WOERKERS, pin_memory=True, pin_memory_device='cuda')
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WOERKERS, pin_memory=True, pin_memory_device='cuda')

In [ ]:
images, labels = next(iter(train_loader))
display(images)
print(labels[:10])

## 2. Build the model <a name="build"></a>

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()

        self.flatten = nn.Flatten()
        self.activate = nn.ReLU()
        
        self.dense_1 = nn.Linear(np.prod(input_size), 200)
        self.dense_2 = nn.Linear(200, 150)
        self.dense_3 = nn.Linear(150, output_size)
    
    def forward(self, inputs):
        x = self.flatten(inputs)
        x = self.dense_1(x)
        x = self.activate(x)
        x = self.dense_2(x)
        x = self.activate(x)
        logits = self.dense_3(x)
        return logits

model = MLP((3, 32, 32), 10).to(device)
model

## 3. Train the model <a name="train"></a>

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

trainer = Trainer(
    model=model, 
    optimizer=optimizer, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    loss_fn=loss_fn,
    device=device,
    metrics=['train_loss', 'accuracy']
)

In [ ]:
history = trainer.fit(epochs=10)

## 4. Evaluation <a name="evaluate"></a>

In [ ]:
trainer.evaluate(test_loader)

In [ ]:
x_test, y_test = map(list, zip(*[test_dataset[i] for i in range(100)]))

CLASSES = np.array(
    [
        "airplane",
        "automobile",
        "bird",
        "cat",
        "deer",
        "dog",
        "frog",
        "horse",
        "ship",
        "truck",
    ]
)

preds = trainer.predict(x_test)
preds_single = CLASSES[preds]
actual_single = CLASSES[y_test]

In [ ]:
n_to_show = 10
indices = np.random.choice(range(len(x_test)), n_to_show)

fig = plt.figure(figsize=(15, 3))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

for i, idx in enumerate(indices):
    img = x_test[idx].permute(1, 2, 0)
    ax = fig.add_subplot(1, n_to_show, i + 1)
    ax.axis("off")
    ax.text(
        0.5,
        -0.35,
        "pred = " + str(preds_single[idx]),
        fontsize=10,
        ha="center",
        transform=ax.transAxes,
    )
    ax.text(
        0.5,
        -0.7,
        "act = " + str(actual_single[idx]),
        fontsize=10,
        ha="center",
        transform=ax.transAxes,
    )
    ax.imshow(img)